In [ ]:
# Version 1 - Mi primer modelo baseline para clasificar SPAM
# Este fue mi punto de partida, sin optimizaciones complejas
# Solo queria ver que tal funcionaba un Bi-LSTM basico
# Resultado: MCC publico 0.8665 - No esta mal para empezar

In [ ]:
# Configuracion inicial del entorno
import os
os.environ['PROTOCOL_BUFFERS_PYTHON_IMPLEMENTATION'] = 'python'  # Fix para Kaggle
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'  # Menos warnings de TensorFlow

# Imports basicos
import pandas as pd  # Manejo de datos
import numpy as np  # Operaciones numericas
import matplotlib.pyplot as plt  # Graficos
import seaborn as sns  # Graficos bonitos
import warnings
warnings.filterwarnings('ignore')  # Nada de warnings

# Semilla para reproducibilidad
seed = 42
np.random.seed(seed)

# TensorFlow y Keras
import tensorflow as tf
from tensorflow import keras
from keras.preprocessing.text import Tokenizer  # Para tokenizar textos
from keras.preprocessing.sequence import pad_sequences  # Para padding
from keras.models import Sequential
from keras.layers import (
    Embedding, LSTM, Bidirectional, Dense, Dropout, 
    GlobalMaxPooling1D, Conv1D, SpatialDropout1D
)
from keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau

tf.random.set_seed(seed)
keras.utils.set_random_seed(seed)

# Metricas de evaluacion
from sklearn.metrics import matthews_corrcoef, classification_report, confusion_matrix
from sklearn.model_selection import train_test_split

pd.set_option('display.max_rows', 36)
pd.set_option("display.max_colwidth", 150)

print("="*60)
print("ENVIRONMENT SETUP")
print("="*60)
print(f"TensorFlow version: {tf.__version__}")
print(f"Keras version: {keras.__version__}")
print(f"Python version: {os.sys.version}")
print(f"GPU Available: {len(tf.config.list_physical_devices('GPU')) > 0}")
if tf.config.list_physical_devices('GPU'):
    print(f"GPU Devices: {tf.config.list_physical_devices('GPU')}")
print(f"Random seed: {seed}")
print("="*60)

## Configuracion del modelo baseline

Hyperparametros iniciales sin mucha optimizacion, solo para probar.

In [ ]:
# Hyperparametros para V1 - Mi primer intento
MAX_WORDS = 10000  # Vocabulario de 10k palabras mas comunes
MAX_LEN = 200  # Secuencias de hasta 200 tokens
EMBEDDING_DIM = 100  # Embeddings de 100 dimensiones

# Arquitectura del modelo
LSTM_UNITS = 128  # Bi-LSTM con 128 unidades (quizas demasiado grande)
DENSE_UNITS = 64  # Capa densa de 64 neuronas
DROPOUT_RATE = 0.5  # Dropout del 50%
SPATIAL_DROPOUT = 0.2  # Spatial dropout en embeddings

# Configuracion de entrenamiento
BATCH_SIZE = 32
EPOCHS = 50  # Hasta 50 epochs pero con early stopping
VALIDATION_SPLIT = 0.2
LEARNING_RATE = 1e-3

print("="*60)
print("MODEL CONFIGURATION")
print("="*60)
print(f"Vocabulary Size: {MAX_WORDS:,}")
print(f"Sequence Length: {MAX_LEN}")
print(f"Embedding Dimension: {EMBEDDING_DIM}")
print(f"LSTM Units: {LSTM_UNITS}")
print(f"Dense Units: {DENSE_UNITS}")
print(f"Dropout Rate: {DROPOUT_RATE}")
print(f"Batch Size: {BATCH_SIZE}")
print(f"Max Epochs: {EPOCHS}")
print("="*60)

## Carga de datos y exploración

In [ ]:
# Cargo el dataset de entrenamiento
train = pd.read_csv("/kaggle/input/u-tad-spam-not-spam-2025-edition/train.csv", index_col="row_id")

print("="*60)
print("TRAINING DATA OVERVIEW")
print("="*60)
print(f"Total samples: {len(train):,}")
print(f"\nColumns: {list(train.columns)}")
print(f"\nData types:\n{train.dtypes}")
print(f"\nNull values:\n{train.isnull().sum()}")
print(f"\nClass distribution:\n{train['spam_label'].value_counts()}")
print(f"\nClass balance:\n{train['spam_label'].value_counts(normalize=True)}")
print("="*60)

# Muestro algunas filas
print("\nSample data:")
train.head(10)

In [ ]:
# Analizo cuanto miden los textos (para decidir MAX_LEN)
train['text_length'] = train['text'].apply(lambda x: len(str(x).split()))

# Grafico la distribucion de longitudes
plt.figure(figsize=(15, 5))

plt.subplot(1, 3, 1)
plt.hist(train['text_length'], bins=50, edgecolor='black', alpha=0.7)
plt.title('Distribución de Longitud de Textos')
plt.xlabel('Número de palabras')
plt.ylabel('Frecuencia')
plt.axvline(train['text_length'].mean(), color='red', linestyle='--', label=f'Media: {train["text_length"].mean():.1f}')
plt.legend()
plt.grid(True, alpha=0.3)

plt.subplot(1, 3, 2)
train.groupby('spam_label')['text_length'].hist(bins=30, alpha=0.7, label=['Not SPAM', 'SPAM'])
plt.title('Longitud por Clase')
plt.xlabel('Número de palabras')
plt.ylabel('Frecuencia')
plt.legend()
plt.grid(True, alpha=0.3)

plt.subplot(1, 3, 3)
sns.boxplot(data=train, x='spam_label', y='text_length')
plt.title('Boxplot Longitud por Clase')
plt.xlabel('Clase (0=Not SPAM, 1=SPAM)')
plt.ylabel('Número de palabras')
plt.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print(f"Longitud promedio: {train['text_length'].mean():.2f} palabras")
print(f"Longitud mediana: {train['text_length'].median():.2f} palabras")
print(f"Longitud máxima: {train['text_length'].max()} palabras")
print(f"Longitud mínima: {train['text_length'].min()} palabras")

## Preprocesamiento - Tokenizacion y padding

In [ ]:
# Preparo textos y etiquetas
X_train_text = train['text'].values
y_train = train['spam_label'].values

# Tokenizo los textos (convierto palabras a numeros)
print("Tokenizando textos...")
tokenizer = Tokenizer(num_words=MAX_WORDS, oov_token="<OOV>")  # OOV = Out Of Vocabulary
tokenizer.fit_on_texts(X_train_text)

# Convierto textos a secuencias de numeros
X_train_seq = tokenizer.texts_to_sequences(X_train_text)

# Padding: hago que todas las secuencias tengan la misma longitud
X_train_pad = pad_sequences(X_train_seq, maxlen=MAX_LEN, padding='post', truncating='post')

# Divido en train y validation
X_train_final, X_val, y_train_final, y_val = train_test_split(
    X_train_pad, y_train, 
    test_size=VALIDATION_SPLIT, 
    random_state=seed,
    stratify=y_train  # Mantengo la proporcion de clases
)

print("="*60)
print("PREPROCESSING SUMMARY")
print("="*60)
print(f"Vocabulary size: {len(tokenizer.word_index):,}")
print(f"Training samples: {len(X_train_final):,}")
print(f"Validation samples: {len(X_val):,}")
print(f"Sequence shape: {X_train_pad.shape}")
print(f"Train class distribution: {np.bincount(y_train_final)}")
print(f"Val class distribution: {np.bincount(y_val)}")
print("="*60)

## Mi modelo V1 - Bi-LSTM baseline

**Arquitectura simple:**
Embedding → SpatialDropout → Bi-LSTM → GlobalMaxPooling → Dense → Dropout → Output

Nada demasiado complejo, solo quiero ver que MCC consigo con un modelo basico.

In [ ]:
# Construccion del modelo V1
def build_model():
    model = Sequential([
        # Embeddings: convierto numeros a vectores densos
        Embedding(
            input_dim=MAX_WORDS,
            output_dim=EMBEDDING_DIM,
            input_length=MAX_LEN,
            name='embedding'
        ),
        
        # Spatial Dropout: dropout pero manteniendo coherencia espacial
        SpatialDropout1D(SPATIAL_DROPOUT),
        
        # Bi-LSTM: Lee el texto en ambas direcciones (izq→der y der→izq)
        Bidirectional(LSTM(LSTM_UNITS, return_sequences=True), name='bidirectional_lstm'),
        
        # GlobalMaxPooling: me quedo con las features mas importantes
        GlobalMaxPooling1D(),
        
        # Capa densa con ReLU
        Dense(DENSE_UNITS, activation='relu', name='dense_1'),
        Dropout(DROPOUT_RATE),  # Dropout del 50% para regularizar
        
        # Salida: sigmoid para probabilidad de SPAM
        Dense(1, activation='sigmoid', name='output')
    ], name='spam_classifier_v1')
    
    return model

# Creo el modelo
model = build_model()

# Compilo con AdamW (Adam con weight decay)
model.compile(
    optimizer=keras.optimizers.AdamW(learning_rate=LEARNING_RATE, weight_decay=1e-4),
    loss='binary_crossentropy',
    metrics=[
        'accuracy',
        keras.metrics.Precision(name='precision'),
        keras.metrics.Recall(name='recall'),
        keras.metrics.AUC(name='auc')
    ]
)

# Veo el resumen
model.summary()

print("\n" + "="*60)
print("MODEL COMPILED")
print("="*60)
print(f"Total parameters: {model.count_params():,}")
print(f"Optimizer: AdamW (lr={LEARNING_RATE}, weight_decay=1e-4)")
print(f"Loss: Binary Crossentropy")
print("="*60)

## Entrenamiento con callbacks

In [ ]:
# Callbacks para controlar el entrenamiento
callbacks = [
    EarlyStopping(
        monitor='val_loss',
        patience=5,  # Si no mejora en 5 epochs, paro
        restore_best_weights=True,
        verbose=1
    ),
    ModelCheckpoint(
        'best_spam_model.keras',
        monitor='val_loss',
        save_best_only=True,  # Solo guardo el mejor modelo
        verbose=1
    ),
    ReduceLROnPlateau(
        monitor='val_loss',
        factor=0.5,  # Reduzco LR a la mitad si no mejora
        patience=3,
        min_lr=1e-6,
        verbose=1
    )
]

print("="*60)
print("STARTING TRAINING")
print("="*60)

# Entreno el modelo
history = model.fit(
    X_train_final, y_train_final,
    batch_size=BATCH_SIZE,
    epochs=EPOCHS,  # Hasta 50 pero early stopping probablemente pare antes
    validation_data=(X_val, y_val),
    callbacks=callbacks,
    verbose=1
)

print("\n" + "="*60)
print("TRAINING COMPLETED")
print("="*60)

## Evaluacion en validation

In [ ]:
# Predigo en validation para ver que tal fue
y_pred_proba = model.predict(X_val)
y_pred = (y_pred_proba > 0.5).astype(int).flatten()

# Calculo el MCC (la metrica de la competicion)
mcc_score = matthews_corrcoef(y_val, y_pred)

print("="*60)
print("VALIDATION METRICS")
print("="*60)
print(f"Matthews Correlation Coefficient: {mcc_score:.4f}")
print("\nClassification Report:")
print(classification_report(y_val, y_pred, target_names=['Not SPAM', 'SPAM']))
print("="*60)

# Matriz de confusion
plt.figure(figsize=(8, 6))
cm = confusion_matrix(y_val, y_pred)
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', 
            xticklabels=['Not SPAM', 'SPAM'],
            yticklabels=['Not SPAM', 'SPAM'])
plt.title(f'Confusion Matrix (MCC: {mcc_score:.4f})')
plt.ylabel('True Label')
plt.xlabel('Predicted Label')
plt.show()

## Curvas de aprendizaje (para detectar overfitting)

In [ ]:
# Visualización de curvas de aprendizaje
def plot_learning_curves(history, title_prefix=""):
    fig, axes = plt.subplots(2, 2, figsize=(15, 10))
    
    # Loss
    axes[0, 0].plot(history.history['loss'], label='Train Loss', linewidth=2)
    axes[0, 0].plot(history.history['val_loss'], label='Val Loss', linewidth=2)
    axes[0, 0].set_title(f'{title_prefix} Loss', fontsize=12, fontweight='bold')
    axes[0, 0].set_xlabel('Epoch')
    axes[0, 0].set_ylabel('Loss')
    axes[0, 0].legend()
    axes[0, 0].grid(True, alpha=0.3)
    
    # Accuracy
    axes[0, 1].plot(history.history['accuracy'], label='Train Accuracy', linewidth=2)
    axes[0, 1].plot(history.history['val_accuracy'], label='Val Accuracy', linewidth=2)
    axes[0, 1].set_title(f'{title_prefix} Accuracy', fontsize=12, fontweight='bold')
    axes[0, 1].set_xlabel('Epoch')
    axes[0, 1].set_ylabel('Accuracy')
    axes[0, 1].legend()
    axes[0, 1].grid(True, alpha=0.3)
    
    # Precision
    axes[1, 0].plot(history.history['precision'], label='Train Precision', linewidth=2)
    axes[1, 0].plot(history.history['val_precision'], label='Val Precision', linewidth=2)
    axes[1, 0].set_title(f'{title_prefix} Precision', fontsize=12, fontweight='bold')
    axes[1, 0].set_xlabel('Epoch')
    axes[1, 0].set_ylabel('Precision')
    axes[1, 0].legend()
    axes[1, 0].grid(True, alpha=0.3)
    
    # Recall
    axes[1, 1].plot(history.history['recall'], label='Train Recall', linewidth=2)
    axes[1, 1].plot(history.history['val_recall'], label='Val Recall', linewidth=2)
    axes[1, 1].set_title(f'{title_prefix} Recall', fontsize=12, fontweight='bold')
    axes[1, 1].set_xlabel('Epoch')
    axes[1, 1].set_ylabel('Recall')
    axes[1, 1].legend()
    axes[1, 1].grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()

plot_learning_curves(history, title_prefix="Model")

# Análisis de overfitting
final_train_loss = history.history['loss'][-1]
final_val_loss = history.history['val_loss'][-1]
final_train_acc = history.history['accuracy'][-1]
final_val_acc = history.history['val_accuracy'][-1]

print("\n" + "="*60)
print("OVERFITTING ANALYSIS")
print("="*60)
print(f"Final Train Loss: {final_train_loss:.4f}")
print(f"Final Val Loss: {final_val_loss:.4f}")
print(f"Loss Difference: {abs(final_val_loss - final_train_loss):.4f}")
print(f"\nFinal Train Accuracy: {final_train_acc:.4f}")
print(f"Final Val Accuracy: {final_val_acc:.4f}")
print(f"Accuracy Difference: {abs(final_val_acc - final_train_acc):.4f}")

if abs(final_val_loss - final_train_loss) < 0.1:
    print("\n✓ Modelo bien balanceado - No hay overfitting significativo")
elif final_val_loss > final_train_loss:
    print("\n⚠ Posible overfitting - Val loss mayor que train loss")
else:
    print("\n⚠ Posible underfitting - Modelo podría mejorar")
print("="*60)

## Predicciones en Test Data

Generación de predicciones para el conjunto de test de la competición.

In [ ]:
# Load test data
test = pd.read_csv("/kaggle/input/u-tad-spam-not-spam-2025-edition/test.csv", index_col="row_id")

print("="*60)
print("TEST DATA")
print("="*60)
print(f"Total test samples: {len(test):,}")
print("="*60)

test.head()

In [ ]:
# Preprocesar test data
X_test_text = test['text'].values
X_test_seq = tokenizer.texts_to_sequences(X_test_text)
X_test_pad = pad_sequences(X_test_seq, maxlen=MAX_LEN, padding='post', truncating='post')

# Generar predicciones
print("Generando predicciones en test data...")
y_pred_proba_test = model.predict(X_test_pad, batch_size=BATCH_SIZE)
y_pred_test = (y_pred_proba_test > 0.5).astype(int).flatten()

print(f"Predicciones generadas: {len(y_pred_test):,}")
print(f"Distribución de predicciones:")
print(f"  Not SPAM (0): {np.sum(y_pred_test == 0):,} ({np.mean(y_pred_test == 0)*100:.2f}%)")
print(f"  SPAM (1): {np.sum(y_pred_test == 1):,} ({np.mean(y_pred_test == 1)*100:.2f}%)")

## Generación del Archivo de Submission

Creación del archivo `submission.csv` para envío a Kaggle.

In [ ]:
# Crear submission file
submission = pd.read_csv("/kaggle/input/u-tad-spam-not-spam-2025-edition/sample_submission.csv")
submission["spam_label"] = y_pred_test
submission.to_csv('submission.csv', index=False)

print("="*60)
print("SUBMISSION FILE CREATED")
print("="*60)
print(f"Total predictions: {len(submission):,}")
print(f"File: submission.csv")
print("="*60)

In [ ]:
# Verificar submission
print("Primeras predicciones:")
submission.head(10)

---

# Análisis Final y Conclusiones

En esta sección se presentan las métricas finales y una reflexión sobre el rendimiento del modelo.

## Resumen de Métricas Finales

In [ ]:
# Tabla resumen de métricas finales
metrics_summary = pd.DataFrame({
    'Métrica': ['MCC', 'Accuracy', 'Precision', 'Recall', 'Loss'],
    'Training': [
        'N/A',  # MCC no se calcula durante entrenamiento
        f"{history.history['accuracy'][-1]:.4f}",
        f"{history.history['precision'][-1]:.4f}",
        f"{history.history['recall'][-1]:.4f}",
        f"{history.history['loss'][-1]:.4f}"
    ],
    'Validation': [
        f"{mcc_score:.4f}",
        f"{history.history['val_accuracy'][-1]:.4f}",
        f"{history.history['val_precision'][-1]:.4f}",
        f"{history.history['val_recall'][-1]:.4f}",
        f"{history.history['val_loss'][-1]:.4f}"
    ]
})

print("="*60)
print("RESUMEN DE MÉTRICAS FINALES - ITERACIÓN 1")
print("="*60)
print(metrics_summary.to_string(index=False))
print("="*60)
print(f"\n📊 Matthews Correlation Coefficient (MCC): {mcc_score:.4f}")
print(f"   → Este es el score que se usará en Kaggle")
print(f"\n🎯 Score Esperado en Kaggle: {mcc_score:.4f} ± 0.02")
print("="*60)

# Visualización comparativa
fig, ax = plt.subplots(figsize=(10, 6))
x = np.arange(len(['Accuracy', 'Precision', 'Recall']))
width = 0.35

train_vals = [
    history.history['accuracy'][-1],
    history.history['precision'][-1],
    history.history['recall'][-1]
]
val_vals = [
    history.history['val_accuracy'][-1],
    history.history['val_precision'][-1],
    history.history['val_recall'][-1]
]

bars1 = ax.bar(x - width/2, train_vals, width, label='Training', alpha=0.8)
bars2 = ax.bar(x + width/2, val_vals, width, label='Validation', alpha=0.8)

ax.set_xlabel('Métrica', fontweight='bold')
ax.set_ylabel('Score', fontweight='bold')
ax.set_title(f'Comparación Métricas - Train vs Validation\nMCC Validation: {mcc_score:.4f}', 
             fontweight='bold', fontsize=12)
ax.set_xticks(x)
ax.set_xticklabels(['Accuracy', 'Precision', 'Recall'])
ax.legend()
ax.set_ylim([0, 1.1])
ax.grid(True, alpha=0.3, axis='y')

# Añadir valores sobre las barras
for bars in [bars1, bars2]:
    for bar in bars:
        height = bar.get_height()
        ax.text(bar.get_x() + bar.get_width()/2., height,
                f'{height:.3f}',
                ha='center', va='bottom', fontsize=9)

plt.tight_layout()
plt.show()

In [ ]:
# Tabla resumen de métricas
metrics_summary = pd.DataFrame({
    'Métrica': [
        'Matthews Correlation Coefficient',
        'Validation Accuracy',
        'Validation Precision',
        'Validation Recall',
        'Training Loss',
        'Validation Loss',
        'Total Parameters',
        'Training Samples',
        'Validation Samples',
        'Test Samples'
    ],
    'Valor': [
        f"{mcc_score:.4f}",
        f"{final_val_acc:.4f}",
        f"{history.history['val_precision'][-1]:.4f}",
        f"{history.history['val_recall'][-1]:.4f}",
        f"{final_train_loss:.4f}",
        f"{final_val_loss:.4f}",
        f"{model.count_params():,}",
        f"{len(X_train_final):,}",
        f"{len(X_val):,}",
        f"{len(test):,}"
    ]
})

print("\n" + "="*60)
print("MÉTRICAS FINALES DEL MODELO")
print("="*60)
print(metrics_summary.to_string(index=False))
print("="*60)

## Reflexión Final sobre el Modelo

### Arquitectura Elegida

El modelo baseline implementado utiliza una arquitectura **LSTM Bidireccional** con las siguientes características clave:

1. **Embeddings**: Capa de embeddings de 100 dimensiones para representar palabras como vectores densos
2. **Spatial Dropout**: Regularización específica para embeddings (20%) que reduce overfitting en la capa de entrada
3. **Bidirectional LSTM**: 128 unidades que capturan contexto tanto hacia adelante como hacia atrás en el texto
4. **Global Max Pooling**: Extrae las características más relevantes de la secuencia completa
5. **Dense Layer**: 64 unidades con activación ReLU y dropout del 50%
6. **Output Layer**: Clasificación binaria con sigmoid

### Justificación de las Decisiones

- **LSTM Bidireccional**: El contexto en ambas direcciones es crucial para detectar SPAM, ya que palabras clave pueden aparecer al principio o al final del mensaje
- **Dropout Alto (50%)**: Necesario para prevenir overfitting dado el tamaño relativamente pequeño del dataset
- **Global Max Pooling**: Más efectivo que Average Pooling para detectar palabras clave específicas de SPAM
- **AdamW Optimizer**: Mejor generalización que Adam estándar gracias al weight decay

### Análisis de Overfitting/Underfitting

Basándose en las curvas de aprendizaje:
- Si la diferencia entre train loss y val loss es **< 0.1**: Modelo bien balanceado
- Si val loss > train loss significativamente: Posible overfitting → Aumentar dropout o reducir complejidad
- Si ambas losses son altas: Underfitting → Aumentar capacidad del modelo

### Próximas Iteraciones Sugeridas

1. **Iteración 2**: Probar embeddings pre-entrenados (GloVe o Word2Vec)
2. **Iteración 3**: Arquitectura híbrida CNN + LSTM para capturar n-gramas y secuencias
3. **Iteración 4**: Transformers ligeros (DistilBERT) para mejor comprensión contextual
4. **Iteración 5**: Ensemble de múltiples modelos para mejorar robustez

### Expectativas de Score

- **Validation MCC**: Indicador directo del score esperado en Kaggle
- **Target**: > 0.85 MCC para estar en el top 25% de la competición
- **Mejoras esperadas**: +0.05-0.10 MCC con embeddings pre-entrenados y optimización de hiperparámetros

### Referencias

- Keras LSTM Documentation: https://keras.io/api/layers/recurrent_layers/lstm/
- Understanding LSTM Networks: http://colah.github.io/posts/2015-08-Understanding-LSTMs/
- Bidirectional RNN: https://keras.io/api/layers/recurrent_layers/bidirectional/
- Matthews Correlation Coefficient: https://scikit-learn.org/stable/modules/generated/sklearn.metrics.matthews_corrcoef.html